# AFAP Anomaly Efficiency Engine — Validation Notebook

## Purpose of This Notebook

This notebook validates the **Anomaly Efficiency Engine**, a Phase 3 locked AFAP engine designed to detect **abnormal year-over-year efficiency changes**, specifically through **Return on Assets (ROA)** behavior.

This notebook confirms that:

1. Ratio-derived inputs are correctly consumed  
2. Year-over-year ROA changes are computed deterministically  
3. Anomaly flags are triggered consistently  
4. Severity classification logic is stable  
5. Output schema is orchestration-ready and human-interpretable


## Engine Context (Conceptual)

**Engine Name:** `anomaly_efficiency_engine`  
**AFAP Phase:** Phase 3 — Locked Engines  

### Signal Type
- Ratio-derived anomaly detection

### Primary Metric
- Return on Assets (ROA)

### Current Scope
- Detects sharp negative ROA changes year-over-year
- Operates strictly on ratio-derived signals

### Planned Future Extension
- Incorporate raw financial statement values (Revenue, Assets, Cash Flow)
- Cross-validate ratio anomalies against absolute financial movements


## Notebook Setup & Environment Configuration

This cell prepares the environment so the notebook can access AFAP engines from the project root.


In [1]:
import pandas as pd
import sys
import os

# Allow imports from the AFAP project root
sys.path.append(os.path.abspath('..'))


## Import Required AFAP Engines

We import:
- `ratio_engine` to generate deterministic ratio inputs
- `anomaly_efficiency_engine` to evaluate efficiency anomalies


In [2]:
from engines.ratio_engine_core import ratio_engine
from engines.anomaly_efficiency_engine import anomaly_efficiency_engine


## Load Cleaned Financial Statement Data

This dataset represents AFAP-validated, cleaned financial statements and serves as the **single source of truth** for ratio computation.


In [3]:
financials = pd.read_csv("../data/cleaned/Kenya_Airways.csv")


## Generate Core Financial Ratios

The **ratio engine** converts raw financial statements into standardized, deterministic ratio outputs.

Key characteristics:
- No interpretation
- No anomaly detection
- Pure computation layer


In [4]:
ratios_core = ratio_engine(financials)


✅ ratio_engine output validated successfully.


## Normalize Ratio Output into a DataFrame

The anomaly efficiency engine expects a tabular structure.

This step ensures:
- Proper sorting by Company and Year
- Clean indexing
- Explicit structural visibility for validation


In [5]:
ratios_core_df = (
    pd.DataFrame(ratios_core)
    .sort_values(["Company", "Year"])
    .reset_index(drop=True)
)


## Flatten Ratio Metrics for Downstream Engines

⚠️ **Critical Preparation Step**

AFAP anomaly engines expect **flat, explicit metric columns**.

This step:
- Extracts ratio metrics from the nested `metrics` field
- Produces a flat table with one column per ratio
- Preserves strict separation between engines


In [6]:
# Flatten ratio metrics into columns
metrics_df = pd.json_normalize(ratios_core_df["metrics"])

ratios_flat = pd.concat(
    [ratios_core_df[["Company", "Year"]], metrics_df],
    axis=1
)


## Validate Flattened Ratio Inputs

Confirm that required columns exist before passing to the anomaly engine.


In [7]:
ratios_flat.columns


Index(['Company', 'Year', 'current_ratio', 'quick_ratio', 'gross_margin',
       'operating_margin', 'net_margin', 'debt_equity', 'interest_coverage',
       'asset_turnover', 'roa', 'roe'],
      dtype='object')

## Execute the Anomaly Efficiency Engine

The anomaly efficiency engine evaluates **year-over-year ROA changes per company** and flags abnormal efficiency deterioration.


In [8]:
anomaly_results = anomaly_efficiency_engine(ratios_flat)


✅ anomaly_efficiency_engine output validated successfully.


## Preview Engine Output Structure

Inspect the first few anomaly records to verify:
- Correct ROA year-over-year computation
- Flag triggering logic
- Severity classification


In [9]:
anomaly_results[:2]


[{'engine': 'anomaly_efficiency_engine',
  'Company': 'Kenya Airways',
  'Year': 2021,
  'metrics': {'roa_yoy': nan},
  'flags': {'roa_shock': False},
  'severity': 'normal',
  'explanation': 'Efficiency metrics stable.'},
 {'engine': 'anomaly_efficiency_engine',
  'Company': 'Kenya Airways',
  'Year': 2022,
  'metrics': {'roa_yoy': 0.5918147333283996},
  'flags': {'roa_shock': False},
  'severity': 'normal',
  'explanation': 'Efficiency metrics stable.'}]

## Inspect Full Structured Engine Output

Convert the complete anomaly output into a DataFrame for structural review.


In [10]:
pd.DataFrame(anomaly_results)


,engine,Company,Year,metrics,flags,severity,explanation
0,anomaly_efficiency_engine,Kenya Airways,2021,{'roa_yoy': nan},{'roa_shock': False},normal,Efficiency metrics stable.
1,anomaly_efficiency_engine,Kenya Airways,2022,{'roa_yoy': 0.5918147333283996},{'roa_shock': False},normal,Efficiency metrics stable.
2,anomaly_efficiency_engine,Kenya Airways,2023,{'roa_yoy': 0.13713289715766708},{'roa_shock': False},normal,Efficiency metrics stable.
3,anomaly_efficiency_engine,Kenya Airways,2024,{'roa_yoy': -1.1329620594083758},{'roa_shock': True},watch,Abnormal efficiency change detected.


## Flatten Anomaly Results for Human Review

Create a client-facing summary view that highlights:
- Severity level
- Triggered anomalies
- Clear explanations


In [11]:
flat = []

for r in anomaly_results:
    anomalies = [
        name.replace("_", " ")
        for name, triggered in r["flags"].items()
        if triggered
    ]

    flat.append({
        "Company": r["Company"],
        "Year": r["Year"],
        "Severity": r["severity"],
        "Anomalies": ", ".join(anomalies) if anomalies else None,
        "Explanation": r["explanation"]
    })

pd.DataFrame(flat)


,Company,Year,Severity,Anomalies,Explanation
0,Kenya Airways,2021,normal,None,Efficiency metrics stable.
1,Kenya Airways,2022,normal,None,Efficiency metrics stable.
2,Kenya Airways,2023,normal,None,Efficiency metrics stable.
3,Kenya Airways,2024,watch,roa shock,Abnormal efficiency change detected.


## Validation Summary

✅ Ratio metrics flattened explicitly  
✅ ROA available as a first-class column  
✅ Year-over-year change calculated correctly  
✅ Severity logic stable and deterministic  
✅ Output schema validated successfully  
✅ Ready for orchestrator integration


## AFAP Architectural Positioning

This notebook demonstrates the **correct AFAP data flow**:

Raw Financials  
→ Ratio Engine (nested metrics)  
→ Orchestrator / Notebook (flattening & preparation)  
→ Anomaly Efficiency Engine (locked detection logic)  
→ Interpretation layer (outside engine boundary)

This preserves determinism, auditability, and composability.
